## Python Performance Tuning
### Setup 

- Install `line_profiler`, using either `pip install line_profiler` or `conda install line_profiler`

### What is profile?

#### Profiling

#### Deterministic profiling
- Monitor all function calls/returns and other events
- Precise timing
- By instrumenting the program - inserting intructions into the program that collect this timing information

#### Statistical profiling 
- Randomly samples the effective instruction pointer
- Less overhead, but less precise

### Why call counts & profiling statistics are useful?
- identify bugs
- identify possible tuning points
- identify "hot loops" that should be carefully optimized
- identify high level system design or algorithm adoption


### Python **standard** profiling modules

#### `cProfile`
- lowest overhead, written in C, may not be as widely available

#### `profile`
- Written in Python, so much higher overhead, but easier to extend

#### Non-standard `line_profiler`, but popular for ipython & juputer-notebook


In [3]:
import cProfile
import re

In [4]:
cProfile.run('re.compile("foo|bar")')

         185 function calls (180 primitive calls) in 0.001 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.001    0.001 <string>:1(<module>)
        1    0.000    0.000    0.001    0.001 re.py:222(compile)
        1    0.000    0.000    0.001    0.001 re.py:278(_compile)
        1    0.000    0.000    0.000    0.000 sre_compile.py:221(_compile_charset)
        1    0.000    0.000    0.000    0.000 sre_compile.py:248(_optimize_charset)
        1    0.000    0.000    0.000    0.000 sre_compile.py:412(_compile_info)
        2    0.000    0.000    0.000    0.000 sre_compile.py:513(isstring)
        1    0.000    0.000    0.000    0.000 sre_compile.py:516(_code)
        1    0.000    0.000    0.001    0.001 sre_compile.py:531(compile)
      3/1    0.000    0.000    0.000    0.000 sre_compile.py:64(_compile)
        3    0.000    0.000    0.000    0.000 sre_parse.py:105(__init__)
        5    0.00

### What does it mean

- The first line: the number of calls that were monitored. Primitive call is call not induced via recursion. 
- The next line, “Ordered by: standard name”: sorting method
  - ncalls: the number of calls
  - tottime: the total time spent in the given function (and excluding time made in calls to sub-functions)
  - percall: is the quotient of tottime divided by ncalls
  - cumtime: is the cumulative time spent in this and all subfunctions (from invocation till exit). This figure is accurate even for recursive functions
  - percall: is the quotient of cumtime divided by primitive calls
  - filename:lineno(function) : provides the respective data of each functio
  
- two numbers in the first column (for example 3/1), it means that the function recursed. The second value is the number of primitive calls and the former is the total number of calls. Note that when the function does not recurse, these two values are the same, and only the single figure is printed.

### From IPython
Use the `%prun` magic command or `%%prun` cell magic command.

### From Command Line

`python -m cProfile [-o output_file] [-s sort_order] filename.py`

### Practise:  Basel problem, approximate $\pi$
$$\pi^2=\sum_{k=1}^\infty \frac{1}{k^2} = 6 \lim_{k -> \infty} (\frac{1}{1^2} + \frac{1}{2^2} + ... + \frac{1}{k^2})$$

In [9]:
### Example 1
def recip_square(i):
    return 1./i**2

def approx_pi(n=10000000):
    val = 0.
    for k in range(1,n+1):
        val += recip_square(k)
    return (6 * val)**.5

### Example 2
def approx_pi2(n=10000000):
    val = 0.
    for k in range(1,n+1):
        val += 1./k**2
    return (6 * val)**.5

In [16]:
%timeit approx_pi()
%prun approx_pi()

1 loop, best of 3: 6.51 s per loop


#### Using `%timeit approx_pi()`
1 loop, best of 3: 6.51 s per loop

#### You will get a pop-up window in the Notebook showing:
 10000004 function calls in 22.087 seconds

   Ordered by: internal time
<pre>
   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
 10000000   15.834    0.000   15.834    0.000 <ipython-input-6-8fdaa89ffea3>:1(recip_square)
        1    6.253    6.253   22.087   22.087 <ipython-input-6-8fdaa89ffea3>:4(approx_pi)
        1    0.000    0.000   22.087   22.087 {built-in method builtins.exec}
        1    0.000    0.000   22.087   22.087 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
</pre>

In [10]:
%prun approx_pi2()

<pre>
4 function calls in 16.854 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1   16.854   16.854   16.854   16.854 <ipython-input-9-28f45ff46d91>:12(approx_pi2)
        1    0.000    0.000   16.854   16.854 {built-in method builtins.exec}
        1    0.000    0.000   16.854   16.854 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
</pre>

In [ ]:
import itertools
import math
# memory overfloat
# def approx_pi3(n=10000000):   
#     powers = list(map(lambda k: 1 << k, range(1, n+1)))
#     acc = list(itertools.accumulate(powers))
#     print(acc[n-1])

def approx_pi3(n=10000000): 
    
    
approx_pi3()

## Part-3 line_profiler

#### The downside of cProfile: 
- cProfile only times explicit function calls, 
   - e.g. `a[large_index_array] = some_other_large_array` never gets explicitly shown by cProfile

#### line_profiler
- load line_profile extension 
  - From ipython/ jupyter-notebook: `%load_ext line_profiler`

In [1]:
%load_ext line_profiler

In [20]:
def primes(n=1000): 
    A = [True] * (n+1)  # declare fix size array, and initizlize
    A[0] = False
    A[1] = False
    for i in range(2, int(n**0.5)+1):
        if A[i]:
            for j in range(i**2, n+1, i):
                A[j] = False

    return [x for x in range(2, n) if A[x]]

In [21]:
%lprun -f primes primes(10000)

#### Showing in pop-up message window
<pre>
Timer unit: 3.95112e-07 s

Total time: 0.189868 s
File: <ipython-input-2-0ccce2a0061a>
Function: primes at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def primes(n=1000): 
     2         1          420    420.0      0.1      A = [True] * (n+1)
     3         1           16     16.0      0.0      A[0] = False
     4         1           10     10.0      0.0      A[1] = False
     5        99          719      7.3      0.1      for i in range(2, int(n**0.5)):
     6        98         1966     20.1      0.4          if A[i]:
     7     17006       247514     14.6     51.5              for j in range(i**2, n+1, i):
     8     16981       215724     12.7     44.9                  A[j] = False
     9                                           
    10         1        14172  14172.0      2.9      return [x for x in range(2, n) if A[x]]
</pre>

- Line # - The line number in the code
- Hits - The number of times that line was executed
- Time - The total amount of time spent executing the line in the timer’s units
- Per Hit - The average amount of time spent executing the line once in the timer’s unit
- % Time - The percentage of time spent on that line relative to the total amount of recorded time spent in the function
- Line Contents - The actual source code of the line

#### Insighte on the profiling result & Possible improment
- use built-in function
- use a fast library such as NumPy

#### Practise: improve the prime function
Use the following test code to check the results:
<pre>
from nose.tools import assert_equal, assert_less
import timeit
assert_equal(primes(1000), faster_primes(1000))
assert_less(timeit.timeit(faster_primes), timeit.timeit(primes))
</pre>

In [26]:
import numpy as np
def faster_primes(n=1000):
    s = np.arange(3, n, 2)
    for m in range(3, int(n ** 0.5)+1, 2): 
        if s[(m-3)/2]: 
            s[(m*m-3)/2::m]=0
    return np.r_[2, s[s>0]].tolist()

#     """ Input n>=6, Returns a array of primes, 2 <= p < n """
#     sieve = np.ones(n/3 + (n%6==2), dtype=np.bool)
#     sieve[0] = False
#     for i in range(int(n**0.5)/3+1):
#         if sieve[i]:
#             k=3*i+1|1
#             sieve[      ((k*k)/3)      ::2*k] = False
#             sieve[(k*k+4*k-2*k*(i&1))/3::2*k] = False
#     return np.r_[2,3,((3*np.nonzero(sieve)[0]+1)|1)].tolist()

In [ ]:
from nose.tools import assert_equal, assert_less
import timeit
assert_equal(primes(1000), faster_primes(1000))
assert_less(timeit.timeit(faster_primes), timeit.timeit(primes))

C:\DevHome\Python\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\DevHome\Python\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
